<a href="https://colab.research.google.com/github/emendozaa22/chatbot_articulo/blob/main/Tarea_viernes_usando_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

instalamos las bibliotecas necesarias, en este caso usaremos langchain

In [ ]:
!pip install langchain_openai langchain_community langchain_text_splitters langchain_chroma pypdf
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/664.8 MB 127.1 MB/s eta 0:00:05ERROR: Operation cancelled by user
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/664.8 MB 109.7 MB/s eta 0:00:06


In [ ]:
import warnings

warnings.filterwarnings('ignore')

from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader
file_path = "/content/2006.11870v1.pdf"  #Ruta al archivo PDF
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))  #Imprimimos la cantidad de partes del PDF

13


In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
#Ruta a la carpeta que contiene los PDFs
folder_path = "/content" #Ajusta esta ruta a tu carpeta

#Lista para almacenar todos los documentos
all_docs = []

#Iteramos sobre todos los archivos en la carpeta

for filename in os.listdir(folder_path):
  #verificamos si el archivo es un pdf
    if filename.endswith(".pdf"):
      #construimos la ruta completa al archivo
        file_path = os.path.join(folder_path, filename)

        try:
           #Cargamos el PDF
            loader = PyPDFLoader(file_path)
            docs = loader.load()
            #Agregamos los documentos a nuestra lista
            all_docs.extend(docs)
            print(f"Procesando: {filename} - {len(docs)} paginas")
        except Exception as e:
            print(f"Error al cargar el archivo {filename}: {str(e)}")
            #Imprimimos el total de paginas procesadas
            print(f"\nTotal de paginas procesadas: {len(all_docs)}")

Procesando: Banca de inversion.pdf - 13 paginas


In [ ]:
import chromadb

persistent_client = chromadb.PersistentClient(path='./vectordb_gratis')

from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwars = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwars,
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits  = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./vectordb_gratis")
retriever = vectorstore.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retriever.get_relevant_documents("smote")

<ipython-input-5-353c33ec514c>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("smote")


[Document(id='e01ecf68-dbe9-42c1-8121-d422e18167e9', metadata={'author': 'Enrique Garcia-Ceja; Michael Riegler; Petter Jakobsen; Jim Torresen; Tine Nordgreen; Ketil J. Oedegaard; Ole Bernt Fasmer', 'creationdate': '2018-06-28T23:07:39+00:00', 'creator': 'PyPDF', 'ieee article id': '8417257', 'ieee issue id': '8417175', 'ieee publication id': '8410976', 'keywords': 'Depression; Bipolar Disorder; Depressive Disorder; Major; Motor Activity; Machine Learning; Artificial Intelligence', 'meeting ending date': '21 June 2018', 'meeting starting date': '18 June 2018', 'moddate': '2022-05-16T12:50:07-04:00', 'page': 3, 'page_label': '4', 'producer': 'OpenPDF 1.0.0-SNAPSHOT; modified using iText® 7.1.16 ©2000-2021 iText Group NV (IEEE; licensed version)', 'source': '/content/2018 Motor_Activity_Based_Classification_of_Depression_in_Uni_and_Bipolar_Patients.pdf', 'subject': '2018 IEEE 31st International Symposium on Computer-Based Medical Systems (CBMS);2018; ; ;10.1109/CBMS.2018.00062', 'title': 

In [ ]:
!pip install colab-xterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 5.8 MB/s eta 0:00:00


In [ ]:

from groq import Groq
import gradio as gr
import os

# Configurar la API key de Groq
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
!pip install langchain langchain-groq

from langchain_groq import ChatGroq

# Configura el modelo de Groq
llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # Puedes cambiar el modelo según tus necesidades
    temperature=0,
    groq_api_key="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"  # Reemplaza con tu clave de API de Groq
)

# Invoca el modelo
respuesta = llm.invoke("")
print(respuesta.content)

Hello! I'm here to help you with any questions you have about writing. What would you like to know?

Here are a few tips to get you started:

1. Read widely and often. This will help you develop your own writing style and expand your vocabulary.
2. Write regularly, even if it's just for a few minutes a day. The more you practice, the better you'll get.
3. Don't be afraid to make mistakes. Writing is a process, and it's okay to revise and edit your work as you go.
4. Show, don't tell. Instead of telling your readers what's happening, use descriptive language to help them visualize the scene.
5. Use active voice instead of passive voice. Active voice makes your writing more engaging and easier to follow.
6. Avoid clichés and overused phrases. Instead, try to come up with fresh, original ways to express your ideas.
7. Write with a clear purpose in mind. Whether you're writing an essay, a story, or a letter, know what you want to say and how you want to say it.
8. Revise and edit your work

In [ ]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
chroma_local = Chroma(persist_directory="./vectordb_gratis", embedding_function=embeddings)

def prompt(texto):
    system_prompt = (
        texto +
        "\n\n"
        "{context}"
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    return prompt

def respuesta(pregunta, llm, chroma_db, prompt):
    retriever = chroma_db.as_retriever()
    chain = create_stuff_documents_chain(llm, prompt)
    rag = create_retrieval_chain(retriever, chain)
    results = rag.invoke({"input": pregunta})
    return results

texto = """Tú eres un asistente para tareas de respuesta a preguntas.
Usa los siguientes fragmentos de contexto recuperado para responder la pregunta.
Si no sabes la respuesta, di que no sabes. Usa un máximo de tres oraciones y mantén la respuesta concisa."""

In [ ]:
if __name__ == '__main__':
    print(respuesta(input('Haz tu pregunta: '), llm, chroma_local, prompt(texto))['answer'])

Haz tu pregunta:  que la banca de inversion?
La banca de inversión se define como una entidad financiera intermediaria que se dedica a la suscripción y venta de activos financieros a inversionistas institucionales, como fondos mutuos, fondos de jubilación y compañías de seguros. Su compensación por la captación de recursos suele adoptar la forma de honorarios y no de ingresos por intereses. La desregulación de la banca de inversión es considerada como la piedra angular de las crisis financieras, como la de los años 30 y la de 2008, y se requiere una regulación clara y transparente para su eficiencia en el futuro.


In [ ]:
def chat_gradio(pregunta):
    return respuesta(pregunta, llm, chroma_local, prompt(texto))["answer"]

iface = gr.Interface(fn=chat_gradio, inputs="text", outputs="text", title="Chat con RAG")
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56695e7ac26d8f9c26.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
